# Computing the correlation between objectives in multi-objective optimization
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes the Pearson correlation coefficients between each pair of objectives. This provides a measure of the conflict among the objectives.

In [1]:
import pandas as pd
import numpy as np
import itertools

### Prepocess data and normalize the objective space

In [2]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_None/OptimalSolutions_NONE_nondominated.txt")

In [3]:
# Hard-coded manipulations to tidy-up sols

#sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (old climate change sets)
sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (climate change, pack forest, Chilean, portfolio opt)
# nothing for some sets (toth mcDill, small sed fire)

In [4]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [5]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses[" MinOwlHabitat"] = 1
#objSenses["PER"] = 0
#objSenses

In [6]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

### Computing the Pearson correlation coefficients

In [7]:
# Convert and normalize the data

sols = maxAllAndNorm(sols,objSenses)

In [8]:
# Compute correlation coefficients for each combination of objectives


for objPair in itertools.combinations(objs,2):
    print(objPair[0] + ", " + objPair[1] + ", " + \
          str(np.corrcoef(sols[objPair[0]],sols[objPair[1]])[0,1]))

 FireHazard,  MinOwlHabitat, -0.295207878376
 FireHazard,  MaxSediment, -0.667934724509
 MinOwlHabitat,  MaxSediment, 0.163242331299
